load resnet50 trained on imagenet50 -> normalize images-> load shap for explainer for image-> analayze the features/pixels of image contributing to classification


In [ ]:
pip install shap tensorflow

In [ ]:
import json
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import shap
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#load pre-trained model and data
model=ResNet50(weights="imagenet")
X,y=shap.datasets.imagenet50()

In [ ]:
print(y)

In [ ]:
plt.imshow(X[20])

In [ ]:
print(X[20])

In [ ]:
#scaling the image
#assume that  image contains data needs to be sacled to range[0,255]
X=np.clip(X,0,255).astype(np.uint8)


In [ ]:
plt.imshow(X[2])

In [ ]:
print(X.shape)

In [ ]:
#getting the 1000 class names
url="https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
with open(shap.datasets.cache(url)) as file:
  class_names=[v[1] for v in json.load(file).values()]

In [ ]:
print("number of imagenet classes:",len(class_names))
print("class names:",class_names)

In [ ]:
def f(x):
    # Preprocess the input x directly, not the global X
    return model.predict(preprocess_input(x))

#define a masker that is used to mask out partitions of the input image
masker=shap.maskers.Image("inpaint_telea",X[0].shape)
#create an explainer with model and image masker
explainer=shap.Explainer(f,masker,output_names=class_names)
#here we explain two images using 1oo evaluation of underlayingmodl to estimate the SHAP values
shap_values=explainer(X[1:3],max_evals=100,batch_size=50,outputs=shap.Explanation.argsort.flip[:4])

In [ ]:
# output with shap values
# this highlighted group of pixels making it look like what predictions

shap.image_plot(shap_values)

# Save the plot as a static image for GitHub compatibility
import matplotlib.pyplot as plt
plt.savefig('shap_image_plot_inpaint.png', bbox_inches='tight')

In [ ]:
def f(x):
    # Preprocess the input x directly, not the global X
    return model.predict(preprocess_input(x))

#define a masker that is used to mask out partitions of the input image
masker=shap.maskers.Image("blur(128,128)",X[0].shape)
#create an explainer with model and image masker
explainer=shap.Explainer(f,masker,output_names=class_names)
#here we explain two images using 1oo evaluation of underlayingmodl to estimate the SHAP values
shap_values_fine=explainer(X[1:3],max_evals=500,batch_size=50,outputs=shap.Explanation.argsort.flip[:4])

In [ ]:
shap.image_plot(shap_values_fine)

# Save the plot as a static image for GitHub compatibility
import matplotlib.pyplot as plt
plt.savefig('shap_image_plot_blur.png', bbox_inches='tight')